We got 16 pages of data in one scrape. We'll go ahead with this as our main dataframe. If necessary we can add another 15 pages of data, for a total of around 3,000 datapoints. 

In [1]:
import pandas as pd
import numpy as np
import bs4
import requests
import neweggutils
import re

from collections import OrderedDict

%load_ext autoreload
%autoreload 2

In [3]:
!ls

1-5_master.csv                      extract-clean-newegg.ipynb
1-5comp.csv                         first_5_components.csv
1-5price.csv                        first_5_components_backup.csv
11_to_14_components.csv             first_5_prices.csv
11_to_14_prices.csv                 first_5_prices_backup.csv
16_comp.csv                         newegg-utils.py
16_price.csv                        neweggutils.py
6-11comp.csv                        prices.csv
6-11price.csv                       scale-scrape.ipynb
6_to_11_component.csv               scrape-amazon.ipynb
6_to_11_components.csv              scrape-newegg.ipynb
6_to_11_price.csv                   scrape-nutrition.ipynb
6_to_11_prices.csv                  scrape-stocks.ipynb
ETL-clean.ipynb                     test_components.csv
README.md                           test_components.html
__pycache__                         test_prices.csv
components.csv                      tibbott-project-luther-proposal.pdf


In [6]:
first5 = pd.read_csv('1-5_master.csv', index_col=0)

In [109]:
sixteen_c = pd.read_csv('16_price.csv')
sixteen_p = pd.read_csv('16_comp.csv')

In [110]:
sixteen_p.head()

,Unnamed: 0,component_html,price_html
0,0,NaN,"<!DOCTYPE HTML>\r\n<html lang=""en"">\r\n<head>\..."
1,0,NaN,"<!DOCTYPE HTML>\r\n<html lang=""en"">\r\n<head>\..."
2,0,NaN,"<!DOCTYPE HTML>\r\n<html lang=""en"">\r\n<head>\..."
3,0,NaN,"<!DOCTYPE HTML>\r\n<html lang=""en"">\r\n<head>\..."
4,0,NaN,"<!DOCTYPE HTML>\r\n<html lang=""en"">\r\n<head>\..."


In [153]:
p16 = extract_load(sixteen_c, sixteen_p)

In [154]:
p16.head()

,price,link,Brand,Series,Model,Part Number,Type,Form Factor,Usage,Colors,...,Package Type,WLAN,Video Memory,L2 Cache Per CPU,PS/2,COM,Installed Qty,LPT,Optical Drive 2,Optical Drive 2 Type
0,579,https://www.newegg.com/Product/Product.aspx?It...,HP,ProDesk,400 G4 (Z2H65UT#ABA),Z2H65UT#ABA,Business Desktops & Workstations,Micro Tower,Business,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1,009",https://www.newegg.com/Product/Product.aspx?It...,HP,EliteDesk,800 G3 (1FY71UT#ABA),1FY71UT#ABA,Business Desktops & Workstations,Tower,Business,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,939,https://www.newegg.com/Product/Product.aspx?It...,HP,NaN,NaN,NaN,NaN,Desktop Computer,NaN,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,985,https://www.newegg.com/Product/Product.aspx?It...,Hewlett-Packard,EliteDesk 800 G3,1FZ01UT#ABA,NaN,Business Desktops & Workstations,Slim/Small form factor,Business,Black/Silver,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,955,https://www.newegg.com/Product/Product.aspx?It...,HP,EliteDesk,800 G3 (1FZ08UT#ABA),1FZ08UT#ABA,Business Desktops & Workstations,Slim/Small form factor,Business,Black,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
def extract_load(components, prices):
    '''
        Parses HTML of two dataframes and merges them in to a single dataframe.
        ====Parameters====
        components: html of individual computer pages
        prices: html of product array page with 96 products
        ====Returns====
        merged: dataframe with prices corresponding to components
    '''
    try:
        prices.drop(labels=['Unnamed: 0', 'component_html'], axis=1, inplace=True)
        components.drop(labels=['Unnamed: 0', 'price_html'], axis=1, inplace=True)

    except (KeyError, ValueError):
        pass
    
    components = components.apply(lambda x: neweggutils.get_components(x[0]), axis=1)
    prices = prices.apply(lambda x: neweggutils.get_prices_and_links(x[0]))# axis=1) if prices is df
    
    components = components.dropna()
    prices = prices.dropna()
    
    components = pd.DataFrame.from_records(components.values)    
    
    # Concatenate all rows in to one list of prices and links
    concat_prices = []
    
    for price in prices:
        concat_prices.extend(price)
                
    prices = pd.DataFrame(concat_prices)
    prices.columns = ['price', 'link']
    
    merged = prices.merge(components, on='link')
    
    return merged

Load pages 16 and on dataframe:

Drop unnecessary columns.

In [99]:
sixteen_c.drop(labels=['Unnamed: 0', 'price_html'], axis=1, inplace=True)
sixteen_p.drop(labels=['Unnamed: 0', 'component_html'], axis=1, inplace=True)

KeyError: "labels ['Unnamed: 0' 'price_html'] not contained in axis"

In [100]:
sixteen_p.drop(labels=['Unnamed: 0', 'component_html'], axis=1, inplace=True)

Parse HTML to get price and corresponding links from `sixteen_p`, and price and corresponding components from `sixteen_c`. We will use the links to join corresponding prices and components.

In [77]:
sixteen_c = sixteen_c.apply(lambda x: neweggutils.get_components(x[0]), axis=1)
sixteen_p = sixteen_p.apply(lambda x: neweggutils.get_prices_and_links(x[0]), axis=1)

In [101]:
sixteen_p = sixteen_p.apply(lambda x: neweggutils.get_prices_and_links(x[0]), axis=1)

In [97]:
sixteen_p.head()

,Unnamed: 0,component_html,price_html
0,0,NaN,"<!DOCTYPE HTML>\r\n<html lang=""en"">\r\n<head>\..."
1,0,NaN,"<!DOCTYPE HTML>\r\n<html lang=""en"">\r\n<head>\..."
2,0,NaN,"<!DOCTYPE HTML>\r\n<html lang=""en"">\r\n<head>\..."
3,0,NaN,"<!DOCTYPE HTML>\r\n<html lang=""en"">\r\n<head>\..."
4,0,NaN,"<!DOCTYPE HTML>\r\n<html lang=""en"">\r\n<head>\..."


In [78]:
sixteen_c = sixteen_c.dropna()

In [79]:
components = pd.DataFrame.from_records(sixteen_c.values)

In [84]:
all_prices = []

for l in sixteen_p:
    all_prices.extend(l)

In [143]:
all_prices

[('579',
  'https://www.newegg.com/Product/Product.aspx?Item=1VK-001E-0AMS1&ignorebbr=1'),
 ('1,009',
  'https://www.newegg.com/Product/Product.aspx?Item=1VK-001E-0AMS3&ignorebbr=1'),
 ('939',
  'https://www.newegg.com/Product/Product.aspx?Item=1B4-001S-005Y6&ignorebbr=1'),
 ('985',
  'https://www.newegg.com/Product/Product.aspx?Item=1B4-001S-005Z3&ignorebbr=1'),
 ('955',
  'https://www.newegg.com/Product/Product.aspx?Item=1B4-001S-005Z2&ignorebbr=1'),
 ('632',
  'https://www.newegg.com/Product/Product.aspx?Item=0N6-00ZF-000Y3&ignorebbr=1'),
 ('1,202',
  'https://www.newegg.com/Product/Product.aspx?Item=0XP-002G-00EM2&ignorebbr=1'),
 ('549',
  'https://www.newegg.com/Product/Product.aspx?Item=1VK-0001-0ZU07&ignorebbr=1'),
 ('499',
  'https://www.newegg.com/Product/Product.aspx?Item=0XP-002G-00EM1&ignorebbr=1'),
 ('599',
  'https://www.newegg.com/Product/Product.aspx?Item=1VK-0001-0ZU05&ignorebbr=1'),
 ('989',
  'https://www.newegg.com/Product/Product.aspx?Item=1VK-001E-0AHR2&ignorebbr=

In [114]:
components = pd.DataFrame(all_prices, columns=['price', 'link'])

In [115]:
components.head()

,price,link
0,579,https://www.newegg.com/Product/Product.aspx?It...
1,"1,009",https://www.newegg.com/Product/Product.aspx?It...
2,939,https://www.newegg.com/Product/Product.aspx?It...
3,985,https://www.newegg.com/Product/Product.aspx?It...
4,955,https://www.newegg.com/Product/Product.aspx?It...
